In [ ]:
import geopandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Datos de establecimientos del sector salud

In [ ]:
file_salud = "data/denue_salud/conjunto_de_datos/denue_inegi_62_.shp"

In [ ]:
sector_salud = geopandas.read_file(file_salud)

In [ ]:
sector_salud.info()

In [ ]:
sector_salud.head()

In [ ]:
sector_salud.plot()

In [ ]:
sector_salud.nombre_act.value_counts()

### Modificamos el tipo de datos para el código

In [ ]:
sector_salud.codigo_act = sector_salud.codigo_act.astype(np.int64)

### Solo tomamos hospitales
Catálogo en: https://www.inegi.org.mx/app/scian/

In [ ]:
hospitales = sector_salud.loc[sector_salud.codigo_act.between(622000, 623000), ]

In [ ]:
hospitales.shape

In [ ]:
hospitales.plot()

In [ ]:
hospitales.nombre_act.value_counts().plot.bar()

### Clasificamos en sector público o privado

In [ ]:
hospitales["sector"] = hospitales["codigo_act"].apply(lambda codigo: 'Sector público' if codigo % 2 == 0 
                                                   else 'Sector privado')

In [ ]:
hospitales_entidad_sector = hospitales.groupby(["entidad", "sector"]).count()["id"].reset_index(name='count')

In [ ]:
hospitales_entidad_sector = hospitales_entidad_sector.pivot(index='entidad', columns='sector', values='count')
hospitales_entidad_sector.head()

In [ ]:
plt.rcParams['figure.figsize'] = [12, 6]
hospitales_entidad_sector.plot.bar(stacked = True)

In [ ]:
hospitales_entidad_sector_percent = hospitales_entidad_sector.apply(lambda x: x*100/sum(x), axis=1)
hospitales_entidad_sector_percent.head()

In [ ]:
hospitales_entidad_sector_percent.plot.bar(stacked = True)

## Incluimos el marco geoestadístico

In [ ]:
file_marco = "data/marco_geoestadistico/CONTINUO_NACIONAL/ESTADOS.shp"
geo_estados = geopandas.read_file(file_marco)

In [ ]:
geo_estados.info()

In [ ]:
geo_estados.plot()

In [ ]:
base = geo_estados.plot(color="white", edgecolor="black")
hospitales.plot(ax=base, marker='o', color="blue", markersize=2)

In [ ]:
geopandas.sjoin(geo_estados, hospitales, how="inner", op='intersects')

In [ ]:
hospitales = hospitales.to_crs(geo_estados.crs)

In [ ]:
base = geo_estados.plot(color="white", edgecolor="black")
hospitales.plot(ax=base, marker='o', color="blue", markersize=2)

In [ ]:
hospitales_estados = hospitales.groupby("cve_ent").count()["id"].reset_index(name='count')
hospitales_estados.columns = ["CVE_ENT", "count"]
hospitales_estados.head()

In [ ]:
geo_estados_hospitales = geo_estados.merge(hospitales_estados, on = "CVE_ENT")

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]
fig, ax = plt.subplots(1, 1)

geo_estados_hospitales.plot(column='count', ax=ax, legend=True)

### Agregamos poblacion

In [ ]:
poblacion = pd.read_csv("data/estructura_00_valor.tsv", sep="\t")
poblacion

In [ ]:
poblacion = pd.read_csv("data/estructura_00_valor.tsv", sep ="\t", header=None,  skiprows=1)
poblacion

In [ ]:
columnames  = pd.read_csv("data/estructura_00_valor.tsv", header=None,  nrows=1)
columnames

In [ ]:
poblacion.columns =  columnames.transpose().iloc[:,0 ].values

In [ ]:
poblacion.head()

In [ ]:
poblacion = poblacion.loc[:,["entidad", "desc_entidad", "municipio", "desc_municipio", "id_indicador", "indicador", 2010]]

In [ ]:
poblacion

In [ ]:
poblacion = poblacion.loc[((poblacion.id_indicador == 1002000002) | (poblacion.id_indicador == 1002000003))]
poblacion

In [ ]:
poblacion = poblacion.loc[(poblacion.municipio == 0) & (poblacion.entidad != 0), ]
poblacion

In [ ]:
poblacion_entidad = poblacion.groupby("entidad").sum()[2010].reset_index(name="poblacion")

In [ ]:
poblacion_entidad["CVE_ENT"] = poblacion_entidad.entidad.map("{:02}".format)

In [ ]:
poblacion_entidad.head()

In [ ]:
geo_estados_hospitales_poblacion = geo_estados_hospitales.merge(poblacion_entidad, on = "CVE_ENT")

In [ ]:
geo_estados_hospitales_poblacion

In [ ]:
geo_estados_hospitales_poblacion["hosp_100k"] = geo_estados_hospitales_poblacion["count"]/(geo_estados_hospitales_poblacion["poblacion"]/100000)

In [1]:
plt.rcParams['figure.figsize'] = [20, 10]
fig, ax = plt.subplots(1, 1)

geo_estados_hospitales_poblacion.plot(column='hosp_100k', ax=ax, legend=True)

NameError: name 'plt' is not defined